# Clean Data File

The JZ Flight School will be evaluating airplane safety by looking at the protection offered by an aircraft during various accidents.  The study will determine how this metric is effected by airplane make, model, phase of flight, and time of year.  

In order to do this, the study will load a database of accidents from the National Transportation Safety Board, and use a variety of features in the study.  To prepare the data, we had to do the following:
- Load the csv file
- Filter for accidents involving one or two engine aircraft
- Normalize manufacturer names and filter for the top 5 aircraft manufacturers
- Imput missing values into the accident statistic column
- Create two new columns: Total.Passengers and Fraction.Fatal

## Import Data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Data/AviationData.csv', encoding='latin1')
df.head()

/var/folders/ym/68nrz1n97wj0gz5413bhpqs80000gn/T/ipykernel_13745/652367376.py:4: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/AviationData.csv', encoding='latin1')


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Filter for Accidents and One and Two Engine Aircrafts 

Only consider events labeled "Accident" and aircraft with one or two engines.

In [2]:
df = df[df['Investigation.Type'] == 'Accident']
df["Investigation.Type"].value_counts()

Investigation.Type
Accident    85015
Name: count, dtype: int64

Our flight school is only looking at airplanes with one or two engines, so we will filter for those.

In [3]:
df = df[(df['Number.of.Engines'] == 1.0) | (df['Number.of.Engines'] == 2.0)]
df["Number.of.Engines"].value_counts()

Number.of.Engines
1.0    69069
2.0     9405
Name: count, dtype: int64

## Normalize the Manufacturer Names

Make certain consistent capitolization is used for manufacturer names.

In [4]:
df['Make'] = df['Make'].str.title()
df['Make'].value_counts().head(20)

Make
Cessna            25864
Piper             14187
Beech              4918
Bell               2350
Mooney             1281
Grumman            1138
Bellanca           1023
Robinson           1012
Hughes              874
Boeing              819
Air Tractor         647
Aeronca             629
Maule               573
Champion            504
Stinson             434
Luscombe            409
Aero Commander      398
Taylorcraft         376
Schweizer           372
North American      364
Name: count, dtype: int64

## Filter for top 5 airplane makers

Our flight school will only purchase from the top 5 manufacturers.  Filter for the top 5 manufactures represented in the data set. 

In [5]:
top_five = df['Make'].value_counts().index[:5]
df = df[df["Make"].map(lambda x:x in top_five)]
df["Make"].value_counts()

Make
Cessna    25864
Piper     14187
Beech      4918
Bell       2350
Mooney     1281
Name: count, dtype: int64

## Check for missing value and create suitable fillin

There are values missing from the injury and fatality statistics.  Imput the missing values with a value of 0.

In [6]:
df["Total.Fatal.Injuries"].fillna(0, inplace=True)
df["Total.Serious.Injuries"].fillna(0, inplace=True)
df["Total.Minor.Injuries"].fillna(0, inplace=True)
df["Total.Uninjured"].fillna(0, inplace=True)

df.head(10).T

,1,2,6,7,8,10,11,13,15,17
Event.Id,20001218X45447,20061025X01555,20001218X45446,20020909X01562,20020909X01561,20020909X01559,20020909X01558,20020917X02134,20020917X02117,20020917X01656
Investigation.Type,Accident,Accident,Accident,Accident,Accident,Accident,Accident,Accident,Accident,Accident
Accident.Number,LAX94LA336,NYC07LA005,CHI81LA106,SEA82DA022,NYC82DA015,FTW82DA034,ATL82DKJ10,FTW82FRA14,FTW82FPG08,ANC82FAG14
Event.Date,1962-07-19,1974-08-30,1981-08-01,1982-01-01,1982-01-01,1982-01-01,1982-01-01,1982-01-02,1982-01-02,1982-01-02
Location,"BRIDGEPORT, CA","Saltville, VA","COTTON, MN","PULLMAN, WA","EAST HANOVER, NJ","HOBBS, NM","TUSKEGEE, AL","HEARNE, TX","LITTLE ROCK, AR","SKWENTA, AK"
Country,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
Latitude,NaN,36.922223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Longitude,NaN,-81.878056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Airport.Code,NaN,NaN,NaN,NaN,N58,NaN,NaN,T72,NaN,NaN
Airport.Name,NaN,NaN,NaN,BLACKBURN AG STRIP,HANOVER,NaN,TUSKEGEE,HEARNE MUNICIPAL,NaN,NaN


## Add new columns: Survive, Total_Passangers, Month

Add new columns survive, total passengers, month, and year:

In [7]:
df['Survive'] = df['Total.Fatal.Injuries'] == 0
df["total.passengers"] = df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"] + df["Total.Minor.Injuries"] + df["Total.Uninjured"]

df['Month'] = (pd.to_datetime(df['Event.Date'])).dt.month
df['Year'] = (pd.to_datetime(df['Event.Date'])).dt.year

df.head().T

,1,2,6,7,8
Event.Id,20001218X45447,20061025X01555,20001218X45446,20020909X01562,20020909X01561
Investigation.Type,Accident,Accident,Accident,Accident,Accident
Accident.Number,LAX94LA336,NYC07LA005,CHI81LA106,SEA82DA022,NYC82DA015
Event.Date,1962-07-19,1974-08-30,1981-08-01,1982-01-01,1982-01-01
Location,"BRIDGEPORT, CA","Saltville, VA","COTTON, MN","PULLMAN, WA","EAST HANOVER, NJ"
Country,United States,United States,United States,United States,United States
Latitude,NaN,36.922223,NaN,NaN,NaN
Longitude,NaN,-81.878056,NaN,NaN,NaN
Airport.Code,NaN,NaN,NaN,NaN,N58
Airport.Name,NaN,NaN,NaN,BLACKBURN AG STRIP,HANOVER


## Add new columns: Fraction_Fatal, Fraction_uninjured

Add new columns fraction fatal, fraction uninjured to the dataframe.

In [10]:
df["Fraction_fatal"] = df["Total.Fatal.Injuries"]/df["total.passengers"]
df["Fraction_uninjured"] = df["Total.Uninjured"]/df["total.passengers"]
df.head().T

,1,2,6,7,8
Event.Id,20001218X45447,20061025X01555,20001218X45446,20020909X01562,20020909X01561
Investigation.Type,Accident,Accident,Accident,Accident,Accident
Accident.Number,LAX94LA336,NYC07LA005,CHI81LA106,SEA82DA022,NYC82DA015
Event.Date,1962-07-19,1974-08-30,1981-08-01,1982-01-01,1982-01-01
Location,"BRIDGEPORT, CA","Saltville, VA","COTTON, MN","PULLMAN, WA","EAST HANOVER, NJ"
Country,United States,United States,United States,United States,United States
Latitude,NaN,36.922223,NaN,NaN,NaN
Longitude,NaN,-81.878056,NaN,NaN,NaN
Airport.Code,NaN,NaN,NaN,NaN,N58
Airport.Name,NaN,NaN,NaN,BLACKBURN AG STRIP,HANOVER


## Write the DataFrame to a CSV file

In [11]:
df.to_csv('Data/AviationDataClean.csv')